In [31]:
from metafid.deriv import Pricing, OptionStrategy
from metafid.data import TSETMC
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 
import finpy_tse as fpy
import psycopg
import jdatetime as jdt
import numpy as np
import re
from CONFIG import DB_PASS as PASS
import inspect

In [33]:
def insert_data(table,cols, records):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            with cur.copy(f"COPY {table} {cols} FROM STDIN") as copy:
                for record in records:
                    copy.write_row(record)
            # Make the changes to the database persistent
            conn.commit()

In [34]:
drop_cols = ["isin","time","open","close","no","volume","low","high","y_final","eps","base_vol","unknown1","unknown2","sector","day_ul","day_ll", "share_no", "mkt_id", "sell_no", "buy_no"]
ua_list = ["شپنا","خودرو","فملي","شستا","وبصادر","فولاد","خگستر","سلام","دي","سرو","خاور","وبملت","خساپا","فخوز","فرابورس","اهرم","حسير","خپارس","های وب","چكاپا","توسن","وتجارت","ذوب","بساما","شبندر","سمگا","وخاور","شتران","كاريس","ت ثاميد","پالايش","لبخند","تپسي","كارا"]

In [35]:
def query_df(cols, table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"SELECT {cols} FROM {table}")
            cur.fetchone()
            conn.commit()
            return pd.DataFrame([i[0] for i in cur], columns=re.findall("[a-z A-Z]+", cols))
sigma_df = query_df("(ua,sigma)", "sigma")

In [36]:
bs = Pricing()
tsetmc = TSETMC(ua=sigma_df.ua, drop_cols=drop_cols)
now = jdt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
now

'1402-01-11 15:15:21'

In [13]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [94]:
df = tsetmc.option_mv()
def option_type(x):
    return "call" if x.startswith("ض") else "put"
df = df.assign(type=df.option.map(option_type))
df = df.assign(ua = df.ua.str.replace("ي", "ی").replace("ك", "ک"))
df = df.merge(sigma_df, on="ua", how="inner")
df["bs"] = df.apply(lambda x: bs.black_scholes(s_0=x["ua_final"], k=x["strike_price"], t=x["t"],sigma=float(x["sigma"]),type_=x["type"]),axis=1)
df["bs"] = df.bs.fillna(0).apply(lambda x: np.nan if x == np.nan else int(x))
call = df[df.type =="call"]
put = df[df.type =="put"]


/opt/homebrew/lib/python3.11/site-packages/metafid/deriv/option.py:41: RuntimeWarning: invalid value encountered in sqrt
  sigma * np.sqrt(t)
/opt/homebrew/lib/python3.11/site-packages/metafid/deriv/option.py:43: RuntimeWarning: invalid value encountered in sqrt
  d_2 = d_1 - sigma * np.sqrt(t)


In [84]:
int(round(1.7,0))

2

In [74]:
call

,option,name,final,value,ob_depth,sell_vol,sell_price,buy_price,buy_vol,dt,...,ua_ob_depth,ua_sell_vol,ua_sell_price,ua_buy_price,ua_buy_vol,ua_dt,t,type,sigma,bs
0,ضخود2036,اختیارخ خودرو-4000-1402/02/13,198,91583000,1,551,291,190,20,1402-01-11 15:41:34,...,1,800000,3582,3582,227900,1402-01-11 15:41:34,33,call,0.025066955258397593,-215.0
1,ضخود2036,اختیارخ خودرو-4000-1402/02/13,198,91583000,4,20,450,0,0,1402-01-11 15:41:34,...,1,800000,3582,3582,227900,1402-01-11 15:41:34,33,call,0.025066955258397593,-215.0
2,ضخود2036,اختیارخ خودرو-4000-1402/02/13,198,91583000,5,50,500,0,0,1402-01-11 15:41:34,...,1,800000,3582,3582,227900,1402-01-11 15:41:34,33,call,0.025066955258397593,-215.0
3,ضخود2036,اختیارخ خودرو-4000-1402/02/13,198,91583000,3,1000,390,170,55,1402-01-11 15:41:34,...,1,800000,3582,3582,227900,1402-01-11 15:41:34,33,call,0.025066955258397593,-215.0
4,ضخود2036,اختیارخ خودرو-4000-1402/02/13,198,91583000,2,100,292,177,77,1402-01-11 15:41:34,...,1,800000,3582,3582,227900,1402-01-11 15:41:34,33,call,0.025066955258397593,-215.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2180,ضخپارس600,اختیارخ خپارس-1200-14020621,603,241145000,2,150,930,622,15,1402-01-11 15:41:34,...,1,0,0,1546,5503158,1402-01-11 15:41:34,165,call,0.04257719247207559,535.0
2181,ضخپارس600,اختیارخ خپارس-1200-14020621,603,241145000,4,30,1100,580,100,1402-01-11 15:41:34,...,1,0,0,1546,5503158,1402-01-11 15:41:34,165,call,0.04257719247207559,535.0
2182,ضخپارس600,اختیارخ خپارس-1200-14020621,603,241145000,1,69,880,635,100,1402-01-11 15:41:34,...,1,0,0,1546,5503158,1402-01-11 15:41:34,165,call,0.04257719247207559,535.0
2183,ضخپارس600,اختیارخ خپارس-1200-14020621,603,241145000,3,100,980,620,100,1402-01-11 15:41:34,...,1,0,0,1546,5503158,1402-01-11 15:41:34,165,call,0.04257719247207559,535.0


In [15]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df


In [16]:
stg = OptionStrategy()

In [17]:
df_cc = stg.covered_call(call)
df_bcs = stg.bull_call_spread(call[(call.ob_depth==1) & (call.sell_price>0)& (call.buy_price>0)])
df_bcs = df_bcs.assign(pct_per_day = round((df_bcs.current_profit /  (abs(df_bcs.max_pot_loss)+df_bcs.ua_final))/df_bcs.t, 4))

/opt/homebrew/lib/python3.11/site-packages/metafid/deriv/strategy.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["strategy"] = df.apply(lambda x: stg(buy=x["ua"], sell=x["symbol"]), axis=1)


In [18]:
df_bcs = df_bcs.sort_values(by="pct_per_day", ascending=False)
for _,i in df_bcs.head(2).iterrows():
    print(i.symbol.buy)

ضخود0106
ضخود0106


In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
call_ = call[(call.sell_price < call.bs) & (call.sell_price >0)]
call_ = call_.assign(pct = round(call.bs/ call.sell_price - 1, 2))
call_.sort_values(by="pct", ascending=False)

In [109]:
class OptionStrategy:
    def __init__(self, call:pd.DataFrame, put:pd.DataFrame) -> None:
            self.call = call
            self.put = put
            self.rep_df = pd.DataFrame(columns=["strategy","option", "ua", "t", "bs", "max_pot_profit", "max_pot_loss", "break_even", "current_profit", "pct_cp", "pct_daily_cp"]) 
    def rep_columns(self, cols):
         return [i for i in cols if i in self.rep_df.columns]

    def covered_call(self):
        """
        خرید داراییِ پایه و فروشِ همزمانِ اختیارِ خرید
        سود-زیان برابر ارزشِ فروش اختیار و تفاوت ارزشِ روزِ دارایی ( در صورت افزایش تا حد قیمتِ اعمال) با ارزشِ خریدِ دارایی است.

        :param df: with (ua, symbol, strike_price, ua_sell_price, buy_price) columns. ua: Underlying Asset
        :return: max_pot_profit: حداکثر سود بالقوه- در شرایطی ایجاد می‌شود که قیمتِ داراییِ پایه در تاریخ سر-رسید از قیمتِ اعمال بالاتر-برابر باشد
                break_even : اگر قیمتِ داراییِ پایه از این قیمت پایین-تر بیاد، معامله ،وارد زیان می‌شود
                pct_max_profit :
                current_profit : بر اساسِ قیمتِ کنونیِ داراییِ پایه محاسبه می‌شود
                pct_current_profit :
        """
        df = self.call[self.call.buy_price > 0]
        stg = namedtuple("CoveredCall", "buy sell")
        df["strategy"] = df.apply(lambda x: str(stg(buy=f'({x["ua"]}:{x["ua_final"]})', sell=x["option"])), axis=1)
        df = df.assign(max_pot_profit=df.strike_price - df.ua_sell_price + df.buy_price)
        df = df.assign(break_even=df.ua_sell_price - df.buy_price)
        df = df.assign(pct_max_profit=df.max_pot_profit / df.break_even * 100)
        df["current_profit"] = df.apply(
            lambda x: min(x["strike_price"], x["ua_sell_price"])
            - x["ua_sell_price"]
            + x["buy_price"],
            axis=1,
        )
        df = df.assign(pct_current_profit=df.current_profit / df.break_even * 100)
        df = df.assign(pct_profit_per_day=df.pct_current_profit / df.t,
                       strategy1 = inspect.stack()[0][3])

        return df[self.rep_columns(df.columns)]

    def married_put(self, df: pd.DataFrame):
        """
        خرید داراییِ پایه و خریدِ همزمان اختیارِ فروش
        سود-زیان  برابرِ اختلافِ ارزشِ روز دارایی (در صورت کاهش تا حد قیمتِ اعمال) با ارزشِ خرید دارایی منهایِ ارزش خرید اختیارِ فروش است.
        :param df: with (ua, symbol, strike_price, ua_sell_price, sell_price) columns. ua: Underlying Asset
        :return: max_pot_profit: حداکثر سود بالقوه- در شرایطی ایجاد می‌شود که قیمتِ داراییِ پایه در تاریخ سر-رسید از قیمتِ اعمال بالاتر-برابر باشد
                break_even : اگر قیمتِ داراییِ پایه از این قیمت پایین-تر بیاد، معامله ،وارد زیان می‌شود
                pct_max_profit :
                current_profit : بر اساسِ قیمتِ کنونیِ داراییِ پایه محاسبه می‌شود
                pct_current_profit :
        """
        df = self.put[self.put.sell_price > 0]
        stg = namedtuple("MarriedPut", "buy buy_")
        df["strategy"] = df.apply(lambda x: stg(buy=x["ua"], buy_=x["option"]), axis=1)
        df = df.assign(max_pot_profit="indefinite")
        df = df.assign(break_even=df.ua_sell_price + df.sell_price)
        df["current_profit"] = df.apply(
            lambda x: max(x["strike_price"], x["ua_sell_price"])
            - x["ua_sell_price"]
            - x["sell_price"],
            axis=1,
        )
        df = df.assign(pct_current_profit=df.current_profit / df.break_even * 100)
        df = df.assign(pct_profit_per_day=df.pct_current_profit / df.t,
                       strategy1 = inspect.stack()[0][3])

        return df[self.rep_columns(df.columns)]
    
    def all_strategy(self):
         return pd.concat([self.rep_df, self.covered_call()])

In [110]:
stg = OptionStrategy(call=call, put=put)
test = stg.all_strategy()

/var/folders/wr/p3grj4bj0mg_d17_nj95wc8c0000gn/T/ipykernel_4751/2266219062.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["strategy"] = df.apply(lambda x: str(stg(buy=f'({x["ua"]}:{x["ua_final"]})', sell=x["option"])), axis=1)


In [111]:
test = test.replace(np.nan, None)
test.head()

,strategy,option,ua,t,bs,max_pot_profit,max_pot_loss,break_even,current_profit,pct_cp,pct_daily_cp
0,"CoveredCall(buy='(خودرو:3543)', sell='ضخود2036')",ضخود2036,خودرو,33,-215,608,None,3392,190,None,None
3,"CoveredCall(buy='(خودرو:3543)', sell='ضخود2036')",ضخود2036,خودرو,33,-215,588,None,3412,170,None,None
4,"CoveredCall(buy='(خودرو:3543)', sell='ضخود2036')",ضخود2036,خودرو,33,-215,595,None,3405,177,None,None
5,"CoveredCall(buy='(خودرو:3543)', sell='ضخود5017')",ضخود5017,خودرو,124,816,428,None,2572,428,None,None
6,"CoveredCall(buy='(خودرو:3543)', sell='ضخود5017')",ضخود5017,خودرو,124,816,322,None,2678,322,None,None


In [112]:
cols_db = "(strategy, option,ua, t, bs, max_pot_profit, max_pot_loss, break_even, current_profit, pct_cp, pct_daily_cp)"

In [113]:
def drop(table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"DELETE FROM {table}")
            conn.commit()
drop("derivs_optionstrategy")

In [114]:
ua_hist_list = list(test.itertuples(index=False, name=None))
insert_data(table="derivs_optionstrategy", cols=cols_db, records=ua_hist_list)